In [16]:
from copy import deepcopy
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_no_action_mask
from src.jss_lite.jss_lite import jss_lite
import numpy as np
import time
class tree_node:
    def __init__(self, action=None,parent=None):
        self.action=action
        self.children=[]
        self.state=None
        self.parent=parent
        self.observation=None
        self.visited=False
        # extra attributes for search, not used till now:
        self.leaf_closed=False
        
class tree_construct:
    def __init__(self):
        # change if you want to load klagenfurt ...
        self.load_klagen=False
        # threshold for values that are as good as the threshold*max_policy
        self.threshold=0.7
        curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
        inst=curr_dir + '/resources/jsp_instances/standard/la02.txt'
        instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
        #self.env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
        
        from wrapper.jssp_wrapper_klagenfurt import jssp_klagenfurt_obs_wrapper
        import JSSEnv
        import gym
        self.env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))

        # create root
        self.root=tree_node()
        # get initial state into root; todo: change for multi instances, create all childs for instances
        self.root.observation=self.env.reset()
        self.root.state=self.env.get_state()
        self.done=False
        self.reward=0
        self.trainer=self.load_trainer()
        # path management
        # open leafs should store the leafs with highest rating
        self.open_leafs=[]
        self.open_leafs.append(self.root)
        self.current_path=self.root
        self.sol_pathes=None
    def load_environment(self,path):
        self.env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[path]))
        self.reset()
    def insert(self,action=None,parent=None):
        parent.children.append(tree_node(action=action,parent=self))
        self.env.set_state(parent.state)
        parent.children[-1].observation,reward,done,info=self.env.step(action)
        parent.children[-1].state=self.env.get_state()
        if done:
            #this goes for klagenfurt jssp
            self.__on_done(reward)
            
    def reset(self):
        self.root=tree_node()
        # get initial state into root; todo: change for multi instances, create all childs for instances
        self.root.observation=self.env.reset()
        self.root.state=self.env.get_state()
        self.done=False
        self.reward=0
        #self.trainer=self.load_trainer()
        # path management
        # open leafs should store the leafs with highest rating
        self.open_leafs=[]
        self.open_leafs.append(self.root)
        self.current_path=self.root
        self.sol_pathes=None

    def __on_done(self,reward):
        self.done=True
        if reward > self.reward:
            self.reward=reward
    def load_checkpoint(self,path):
        self.trainer.load_checkpoint(path)
    def load_trainer(self):
        from ray.rllib.agents.ppo import PPOTrainer
        from ray import tune
        import gym
        from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
        from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
        from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_no_action_mask
        from src.jss_lite.jss_lite import jss_lite
        from ray import tune

        if self.load_klagen:
            from wrapper.jssp_wrapper_klagenfurt import jssp_klagenfurt_obs_wrapper
            import JSSEnv
            #Configure the algorithm.
            def env_creator(env_config):
                import JSSEnv
                env = jssp_light_obs_wrapper_no_action_mask(jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/la01.txt'})))
                return env
        else:
            curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
            inst=curr_dir + '/resources/jsp_instances/standard/la01.txt'
            instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
            def env_creator(env_config):
                #env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
                env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
                #env=jss_lite(instance_path=inst)
                #env=jssp_light_obs_wrapper_no_action_mask(jss_lite(instance_path=inst))
                return env

        tune.register_env('custom_jssp',env_creator)
        config = {
            "env": "custom_jssp",
            "disable_env_checking":True,
            "num_workers": 4,
            "framework": "tf",
            "model": {
                "fcnet_hiddens": [128, 128],
                "fcnet_activation": "relu",
            },
            "evaluation_num_workers": 0,
            "evaluation_config": {
                "render_env": False,
            },
            
        }
        trainer = PPOTrainer(config=config)
        
        if self.load_klagen:
            trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-60')
        else:
            trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_punish2ima_8_8_ppo/checkpoint-194')
        return trainer

    def propose_actions(self,state,observation,threshold=None):
        if threshold !=None:
            self.threshold=threshold
        policy = self.trainer.get_policy()
        action, _, info = policy.compute_single_action(observation)
        logits = info['action_dist_inputs']
        self.env.set_state(state=state)
        action_mask = self.env.get_legal_actions().astype(np.float32)
        action_mask[action_mask==0] = - np.inf
        probs=np.multiply(logits,action_mask)
        probs[probs==np.inf]=-np.inf
        best_action_id = probs.argmax()  # deterministic
        highest_prob=probs.max()
        #todo: check if it is good here
        if highest_prob>0:
            out,=np.where(probs>self.threshold*highest_prob)
        else:
            out,=np.where(probs>highest_prob/self.threshold)

        return out, best_action_id    


    def solve(self,max_pathes=None,max_time=None,s_threshold=None):
        # here comes the solving method with abort conditions
        start_time=time.time()
        p_time=time.time()
        self.sol_pathes=0

        if max_pathes==None:
            max_pathes=2 
        if max_time==None:
            max_time=20
        #print(self.open_leafs)
        while self.open_leafs:
            self.sol_pathes+=1
            # abort conditions:
            node=self.open_leafs.pop()
            if self.sol_pathes > max_pathes or time.time()-start_time > max_time or self.reward==1:
                return True
            while not self.done:
                loop=0
                
                #possible_actions=np.random.choice(len(node.observation['action_mask']), p_r, p=(node.observation['action_mask'] / node.observation['action_mask'].sum()))
                possible_actions, best_action = self.propose_actions(node.state,node.observation,threshold=s_threshold)
                for action in set(possible_actions):
                    if action!=best_action:
                        #print(f"action is: {action}")
                        self.insert(action=action,parent=node)
                #assure that best action is insertet last
                self.insert(action=best_action,parent=node)
                for child in node.children:
                    if child != node.children[-1]:
                        #todo: check here if in action mask
                        self.open_leafs.append(child)
                while True:
                    if node.children[-1].visited==False:
                        node.children[-1].visited=True
                        node=node.children[-1]
                        
                        break
                    loop+=1
                    if loop > len(node.children):
                        print("all childrens visited; return done equal true")
                        self.done=True
            self.done=False
            




In [29]:
import ray
ray.shutdown()
init=tree_construct()
print("solve")
init.solve()
print(init.reward)
print(init.sol_pathes)

2022-12-31 17:52:00,579	INFO trainable.py:159 -- Trainable.setup took 24.280 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


solve
50.150150150150154
3


In [33]:
init.reset()
init.solve(max_pathes=1,max_time=30,s_threshold=0.9)
print(init.reward)
print(init.sol_pathes)

68.76876876876877
1001


In [17]:
import ray
ray.shutdown()
init=tree_construct()


2022-12-31 18:38:38,629	INFO trainable.py:159 -- Trainable.setup took 24.188 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [26]:
import pandas as pd
sizes=[3,6,8]
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'

#times={'3':,'6','8':,'10':,'15':}
#checkpoints=['/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_punish_v4_t_tima_'+i+'_'+i+'_ppo' for i in sizes]
for num_inst in sizes:
    results={}
    num_inst=str(num_inst)
    checkpoint='/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_punish_v4_t_tima_'+num_inst+'_'+num_inst+'_ppo/checkpoint-150' 
    init.load_checkpoint(checkpoint)
    print(checkpoint)
    test_instances=[]
    for i in range(20,40):
        test_instances.append(curr_dir+'/resources/jsp_instances/ima/'+num_inst+'x'+num_inst+'x'+num_inst+'/'+num_inst+'x'+num_inst+'_'+str(i)+'_inst.json')
    for nr_i,inst in enumerate(test_instances):
        #print(init.env.env.env.instance)
        init.reset()
        init.load_environment(inst)
        init.solve(max_pathes=10000000,max_time=30,s_threshold=0.5)
        results['instance_'+str(nr_i)]={'reward':init.reward,'number_pathes':init.sol_pathes}
        #print(init.reward)
        #print(init.sol_pathes)
    df=pd.DataFrame.from_dict(results)
    df.to_csv(f"eval_tree_search_{num_inst}_inst_20_40_30sec_t05.csv")


/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_punish_v4_t_tima_3_3_ppo/checkpoint-150
/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_punish_v4_t_tima_6_6_ppo/checkpoint-150
/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_punish_v4_t_tima_8_8_ppo/checkpoint-150


In [26]:

from ray.rllib.agents.ppo import PPOTrainer
from ray import tune
import ray
import gym
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_no_action_mask
from src.jss_lite.jss_lite import jss_lite
from ray import tune
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
inst=curr_dir + '/resources/jsp_instances/standard/la01.txt'
instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
# Configure the algorithm.
def env_creator(env_config):
    #env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
    env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
    #env=jss_lite(instance_path=inst)
    #env=jssp_light_obs_wrapper_no_action_mask(jss_lite(instance_path=inst))
    return env
from wrapper.jssp_wrapper_klagenfurt import jssp_klagenfurt_obs_wrapper
import JSSEnv
# #Configure the algorithm.
# def env_creator(env_config):
#     import JSSEnv
#     env = jssp_light_obs_wrapper_no_action_mask(jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/la01.txt'})))
#     return env

ray.shutdown()
# use tune to register the custom environment for the ppo trainer
tune.register_env('custom_jssp',env_creator)


config = {
    "env": "custom_jssp",
    "disable_env_checking":True,
    "num_workers": 4,
    "framework": "tf",
    "model": {
        "fcnet_hiddens": [256, 256],
        "fcnet_activation": "relu",
    },
    "evaluation_num_workers": 0,
    "evaluation_config": {
        "render_env": False,
    },
    
}
trainer = PPOTrainer(config=config)
#trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/ppo/checkpoint-4')
#trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/klagen_ppo/checkpoint-50')

for _ in range(60):
    print(_)
    trainer.train()
    #trainer.save_checkpoint("training_checkpoints/klagen_ppo")
    trainer.save_checkpoint("training_checkpoints/custom_ppo")




2022-10-27 12:18:59,371	INFO trainable.py:159 -- Trainable.setup took 14.187 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


In [20]:
from ray.rllib.agents.ppo import PPOTrainer
from ray import tune
import ray
import gym
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_no_action_mask
from src.jss_lite.jss_lite import jss_lite
from ray import tune
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
inst='/Users/felix/sciebo/masterarbeit/progra/model-based_rl/resources/jsp_instances/ima/3x3x3/3x3_4_inst.json'#curr_dir + '/resources/jsp_instances/standard/la01.txt'
# Configure the algorithm.
def env_creator(env_config):
    env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
    return env
from wrapper.jssp_wrapper_klagenfurt import jssp_klagenfurt_obs_wrapper
import JSSEnv
ray.shutdown()
tune.register_env('custom_jssp',env_creator)
config = {
    "env": "custom_jssp",
    "disable_env_checking":True,
    "num_workers": 4,
    "framework": "tf",
    "model": {
        "fcnet_hiddens": [128, 128],
        "fcnet_activation": "relu",
    },
    "evaluation_num_workers": 0,
    "evaluation_config": {
        "render_env": False,
    },
    
}
trainer = PPOTrainer(config=config)

env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=['/Users/felix/sciebo/masterarbeit/progra/model-based_rl/resources/jsp_instances/ima/3x3x3/3x3_4_inst.json']))
for _ in range(1,90):
    #print('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_punish_v2ima_8_8_ppo/checkpoint-'+str(_))
    trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_no_punish_v2ima_3_3_ppo/checkpoint-'+str(_))
    import numpy as np
    #trainer.save_checkpoint("ppo")
    episode_reward = 0
    #env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
    #env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
    #env=env_creator("s")
    #env=jssp_light_obs_wrapper_no_action_mask(jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/la01.txt'})))
    done = False
    obs = env.reset()
    iterations=0


    while not done:
        policy = trainer.get_policy()
        action, _lk, info = policy.compute_single_action(obs)
        logits = info['action_dist_inputs']
        action_mask = env.get_legal_actions().astype(np.float32)
        action_mask[action_mask==0] = - np.inf
        probs=np.multiply(logits,action_mask)
        probs[probs==np.inf]=-np.inf
        best_action_id = probs.argmax()  # deterministic
        #best_action_id = logits.argmax()
        #best_action_id = trainer.compute_single_action(obs)
        obs, reward, done, info = env.step(best_action_id)
        episode_reward += reward
        iterations += 1

        #print(best_action_id)
    #env.env.env.render()
    #print(iterations)
    #print(reward)
    print(f"in iteration{_} reward is: {reward}")
    #print(env.env.env.last_time_step-666)

2022-12-30 17:57:24,145	INFO trainable.py:159 -- Trainable.setup took 22.246 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


in iteration1 reward is: 0.6818181818181819
in iteration2 reward is: 0.8636363636363636
in iteration3 reward is: 1.0
in iteration4 reward is: 0.6818181818181819
in iteration5 reward is: 0.6818181818181819
in iteration6 reward is: 0.9090909090909091
in iteration7 reward is: 0.6818181818181819
in iteration8 reward is: 0.6818181818181819
in iteration9 reward is: 0.6818181818181819
in iteration10 reward is: 0.9090909090909091
in iteration11 reward is: 1.0
in iteration12 reward is: 1.0
in iteration13 reward is: 1.0
in iteration14 reward is: 1.0
in iteration15 reward is: 1.0
in iteration16 reward is: 1.0
in iteration17 reward is: 1.0
in iteration18 reward is: 1.0
in iteration19 reward is: 1.0
in iteration20 reward is: 1.0
in iteration21 reward is: 1.0
in iteration22 reward is: 1.0
in iteration23 reward is: 1.0
in iteration24 reward is: 1.0
in iteration25 reward is: 1.0
in iteration26 reward is: 1.0
in iteration27 reward is: 1.0
in iteration28 reward is: 1.0
in iteration29 reward is: 1.0
in i

In [18]:

env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=['/Users/felix/sciebo/masterarbeit/progra/model-based_rl/resources/jsp_instances/ima/8x8x8/8x8_5_inst.json']))
for _ in range(1,90):
    #print('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_punish_v2ima_8_8_ppo/checkpoint-'+str(_))
    trainer.load_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/oneenvironment_punish_v2ima_8_8_ppo/checkpoint-'+str(_))
    import numpy as np
    #trainer.save_checkpoint("ppo")
    episode_reward = 0
    #env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
    #env=jssp_light_obs_wrapper_no_action_mask(jssp_light_obs_wrapper_multi_instances(instances_list=[inst]))
    #env=env_creator("s")
    #env=jssp_light_obs_wrapper_no_action_mask(jssp_klagenfurt_obs_wrapper(gym.make('jss-v1', env_config={'instance_path': 'resources/jsp_instances/standard/la01.txt'})))
    done = False
    obs = env.reset()
    iterations=0


    while not done:
        policy = trainer.get_policy()
        action, _lk, info = policy.compute_single_action(obs)
        logits = info['action_dist_inputs']
        action_mask = env.get_legal_actions().astype(np.float32)
        action_mask[action_mask==0] = - np.inf
        probs=np.multiply(logits,action_mask)
        probs[probs==np.inf]=-np.inf
        best_action_id = probs.argmax()  # deterministic
        #best_action_id = logits.argmax()
        #best_action_id = trainer.compute_single_action(obs)
        obs, reward, done, info = env.step(best_action_id)
        episode_reward += reward
        iterations += 1

        #print(best_action_id)
    #env.env.env.render()
    #print(iterations)
    #print(reward)
    print(f"in iteration{_} reward is: {reward}")
    #print(env.env.env.last_time_step-666)

in iteration1 reward is: 0.5217391304347826
in iteration2 reward is: 0.6086956521739131
in iteration3 reward is: 0.5652173913043479
in iteration4 reward is: 0.6231884057971014
in iteration5 reward is: 0.536231884057971
in iteration6 reward is: 0.6376811594202898
in iteration7 reward is: 0.7246376811594203
in iteration8 reward is: 0.6086956521739131
in iteration9 reward is: 0.7101449275362319
in iteration10 reward is: 0.7391304347826086
in iteration11 reward is: 0.5797101449275363
in iteration12 reward is: 0.6086956521739131
in iteration13 reward is: 0.5217391304347826
in iteration14 reward is: 0.536231884057971
in iteration15 reward is: 0.7971014492753623
in iteration16 reward is: 0.6086956521739131
in iteration17 reward is: 0.6666666666666667
in iteration18 reward is: 0.5942028985507246
in iteration19 reward is: 0.4492753623188406
in iteration20 reward is: 0.6521739130434783
in iteration21 reward is: 0.5942028985507246
in iteration22 reward is: 0.6086956521739131
in iteration23 reward

In [16]:

needed_time=env.env.env.last_time_step
optimal_value=655
reward=1-((needed_time-optimal_value)/optimal_value)
print(reward)
#print(datetime.datetime.fromtimestamp(env.env.env.start_timestamp+env.env.env.last_time_step))
#print((env.env.env.start_timestamp))
#print(datetime.datetime.fromtimestamp(env.env.env.start_timestamp))

#print(env.env.env.last_time_step)
#print(datetime.datetime.fromtimestamp(env.env.env.last_time_step))
#print(10*60*60-9*60*60+6*60-54*60+56-2)



0.6931297709923664


In [ ]:
env.env.env.render()

In [ ]:
env_2=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
x,=env_2.observation_space['obs'].shape
print(x)

In [ ]:
print(iterations)
env.render()

In [ ]:
print(info)
print(len(info['action_dist_inputs']))

In [ ]:
import numpy as np
import time
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from src.jss_lite.jss_lite import jss_lite
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
inst=curr_dir + '/resources/jsp_instances/standard/ft06.txt'
instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
# Configure the algorithm.
def env_creator(env_config):
    env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
    #env=jss_lite(instance_path=inst)
    return env
# get random actions:
reward=0
while reward <0.95:
#for _ in range(100):
    action_list=[]
    env = env_creator("s")

    obs = env.reset()
    # env2 is copy for later going evaluation
    counter=0
    done = False
    t=time.time()
    while not done:
        legal_action=obs['action_mask']
        action=np.random.choice(len(legal_action), 1, p=(legal_action / legal_action.sum()))[0]
        action_list.append(action)
        #print(action)
        #print(obs['action_mask'][action])
        obs, reward, done, _ = env.step(action)
        counter+=1
    #print(f"instance: {env.env.instance} in time: {time.time()-t}s")
    
    #env.render()
    #env.close()
print(counter)
print(reward)
env.render()

In [ ]:
import numpy as np
x=np.array([2,2,2,2,2,2,2,2,2,2,2,2,4])
print(x[x>=0.8*2].argmax())
y,=(np.where(x>=2))

for i in y:
    print(i)